<a href="https://colab.research.google.com/github/shadab-entrepreneur/DataScience/blob/main/Projects/ICLR%20Workshop%20Challenge%20%231%3A%20CGIAR%20Computer%20Vision%20for%20Crop%20Disease/ICLR_crop_diseases_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install --upgrade torch torchvision

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.4.0)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.5.0)


In [0]:
import numpy as np
import re
import pickle
import os
import glob
import pandas as pd
from google.colab import drive
import cv2
from PIL import Image
from pathlib import Path
import fastai
from fastai.vision import *
import torch
torch.cuda.empty_cache()

In [0]:
fastai.__version__

'1.0.60'

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.listdir()

['.config', 'drive', 'sample_data']

In [0]:
# os.mkdir("drive/My Drive/Crop_Disease")
# os.mkdir("drive/My Drive/Crop_Disease/test")
# os.mkdir("drive/My Drive/Crop_Disease/train")

In [0]:
# with zipfile.ZipFile('drive/My Drive/crop_disease.zip') as file: # opening the zip file using 'zipfile.ZipFile' class
#             file.extractall("drive/My Drive/Crop_Disease")

In [0]:
os.chdir('/content/drive/My Drive/Crop_Disease')

In [0]:
os.listdir()

['__MACOSX', 'test', 'train', 'submission.csv', 'submission1.csv']

In [0]:
# with zipfile.ZipFile('train.zip') as file: # opening the zip file using 'zipfile.ZipFile' class
#             file.extractall("train")

# with zipfile.ZipFile('test.zip') as file: # opening the zip file using 'zipfile.ZipFile' class
#             file.extractall("test")

In [0]:
# Fast AI doesn't read in .jfif by default, so we need this.
!sudo apt install imagemagick
!cd 'test/test'; mogrify -format jpg *.jfif
!cd 'train/train/healthy_wheat'; mogrify -format jpg *.jfif  
!cd 'train/train/leaf_rust'; mogrify -format jpg *.jfif  
!cd 'train/train/stem_rust'; mogrify -format jpg *.jfif

Reading package lists... Done
Building dependency tree       
Reading state information... Done
imagemagick is already the newest version (8:6.9.7.4+dfsg-16ubuntu6.8).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
healthy_wheat=glob.glob('train/train/healthy_wheat/*.*')
leaf_rust=glob.glob('train/train/leaf_rust/*.*')
stem_rust=glob.glob('train/train/stem_rust/*.*')

In [0]:
img_path=[]
img_path=img_path+healthy_wheat+leaf_rust+stem_rust
df=pd.DataFrame({'img_path':img_path})
df.shape

(1189, 1)

In [0]:
df.head()

,img_path
0,train/train/healthy_wheat/2KS7HG.jpg
1,train/train/healthy_wheat/6T5756.jfif
2,train/train/healthy_wheat/E5HXTR.jpg
3,train/train/healthy_wheat/R1GKEP.jfif
4,train/train/healthy_wheat/3OKS1S.jfif


In [0]:
# df['img_path'] = df['img_path'].apply(lambda x: re.sub("/","",x,-1))
# df.head()

In [0]:
df['disease_class'] = df['img_path'].apply(lambda x: x.split("/")[2])
df['ext'] = df['img_path'].apply(lambda x: x.split(".")[-1])
df.head()

,img_path,disease_class,ext
0,train/train/healthy_wheat/2KS7HG.jpg,healthy_wheat,jpg
1,train/train/healthy_wheat/6T5756.jfif,healthy_wheat,jfif
2,train/train/healthy_wheat/E5HXTR.jpg,healthy_wheat,jpg
3,train/train/healthy_wheat/R1GKEP.jfif,healthy_wheat,jfif
4,train/train/healthy_wheat/3OKS1S.jfif,healthy_wheat,jfif


In [0]:
df=df[df.ext!='jfif']
df.drop('ext',axis=1,inplace=True)
df.head()

,img_path,disease_class
0,train/train/healthy_wheat/2KS7HG.jpg,healthy_wheat
2,train/train/healthy_wheat/E5HXTR.jpg,healthy_wheat
9,train/train/healthy_wheat/UOZCDJ.jpg,healthy_wheat
12,train/train/healthy_wheat/QJI3IC.jpg,healthy_wheat
16,train/train/healthy_wheat/I9JOL9.jpg,healthy_wheat


In [0]:
df.shape

(876, 2)

In [0]:
df.reset_index(inplace=True)

In [0]:
df.drop('index',axis=1,inplace=True)

In [0]:
df.head()

,img_path,disease_class
0,train/train/healthy_wheat/2KS7HG.jpg,healthy_wheat
1,train/train/healthy_wheat/E5HXTR.jpg,healthy_wheat
2,train/train/healthy_wheat/UOZCDJ.jpg,healthy_wheat
3,train/train/healthy_wheat/QJI3IC.jpg,healthy_wheat
4,train/train/healthy_wheat/I9JOL9.jpg,healthy_wheat


In [0]:
test_img_path=glob.glob('test/test/*.*')
test_df=pd.DataFrame({'test_img_path':test_img_path})
test_df['ext'] = test_df['test_img_path'].apply(lambda x: x.split(".")[-1])
test_df=test_df[test_df.ext!='jfif']
test_df.shape

(610, 2)

In [0]:
test_df.head()

,test_img_path,ext
1,test/test/NGZ2X2.JPG,JPG
2,test/test/AD54KD.jpg,jpg
3,test/test/3UZ447.JPG,JPG
6,test/test/NN5F6I.jpg,jpg
8,test/test/LE4WVZ.JPG,JPG


In [0]:
# test_df['test_img_path'] = test_df['test_img_path'].apply(lambda x: re.sub("/","",x,1))
test_df.drop('ext',axis=1,inplace=True)
test_df.reset_index(inplace=True)
test_df.drop('index',axis=1,inplace=True)
test_df.head()

,test_img_path
0,test/test/NGZ2X2.JPG
1,test/test/AD54KD.jpg
2,test/test/3UZ447.JPG
3,test/test/NN5F6I.jpg
4,test/test/LE4WVZ.JPG


In [0]:
df['disease_class'].value_counts()

stem_rust        376
leaf_rust        358
healthy_wheat    142
Name: disease_class, dtype: int64

In [0]:
def create_databunch(valid_idx):
  test = ImageList.from_df(test_df,path='')
  data = (ImageList.from_df(df, path='')
              .split_by_idx(valid_idx)
              .label_from_df(cols='disease_class')
              .add_test(test)
              .transform(get_transforms(flip_vert=True, max_rotate=20.0), size=128)
              .databunch(path='.', bs=32)
              .normalize(imagenet_stats)
            )
  return data

In [0]:
model_densenet = models.densenet201

In [40]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3,shuffle=True,random_state=123)
acc_val=[]
for train_idx, valid_idx in kf.split(df):
      data = create_databunch(valid_idx)
      learn = cnn_learner(data, model_densenet, metrics=[accuracy])
      learn.fit_one_cycle(2, 1e-3)
      learn.unfreeze()
      learn.fit_one_cycle(10, slice(1e-4,1e-2))
      loss,acc=learn.validate()
      acc_val.append(acc.numpy())

print("The validation accuracy is: ",np.mean(acc_val))

epoch,train_loss,valid_loss,accuracy,time
0,1.505163,0.842372,0.681507,01:35
1,1.231134,0.748997,0.705479,01:38


epoch,train_loss,valid_loss,accuracy,time
0,1.030407,0.796715,0.729452,01:51
1,0.966965,1.153291,0.681507,01:58
2,0.902816,0.980250,0.743151,01:55
3,0.858118,0.471841,0.852740,01:55
4,0.774111,0.474488,0.832192,01:57
5,0.680230,2.781417,0.763699,01:56
6,0.663839,0.693885,0.811644,01:53
7,0.595726,0.360971,0.876712,01:56
8,0.517920,0.307746,0.890411,01:55
9,0.438606,0.284756,0.893836,01:54


epoch,train_loss,valid_loss,accuracy,time
0,1.686260,0.755749,0.719178,01:36
1,1.302094,0.726640,0.750000,01:38


epoch,train_loss,valid_loss,accuracy,time
0,1.027032,1.000369,0.750000,01:55
1,1.020555,1.170388,0.732877,01:55
2,0.976005,1.674768,0.671233,01:54
3,0.854712,0.860845,0.787671,01:53
4,0.769494,0.587097,0.825342,01:56
5,0.689539,0.689372,0.773973,01:52
6,0.561540,0.529100,0.839041,01:55
7,0.473063,0.524675,0.839041,01:56
8,0.383124,0.477837,0.863014,01:53
9,0.321763,0.471473,0.863014,01:56


epoch,train_loss,valid_loss,accuracy,time
0,1.542285,0.970496,0.681507,01:39
1,1.255145,0.793003,0.732877,01:35


epoch,train_loss,valid_loss,accuracy,time
0,1.053627,0.929377,0.722603,01:54
1,1.025459,2.214958,0.613014,01:50
2,0.906407,0.910887,0.773973,01:54
3,0.753774,0.872099,0.815068,01:50
4,0.672999,0.698962,0.784247,01:52
5,0.589701,0.650128,0.787671,01:53
6,0.484638,0.571635,0.845890,01:54
7,0.404793,0.531298,0.869863,01:54
8,0.324395,0.520559,0.876712,01:52
9,0.275423,0.521503,0.869863,01:51


The validation accuracy is:  0.8755708


In [41]:
#Prediction of the Test Set
preds_test, y_test = learn.get_preds(DatasetType.Test)

In [42]:
submission = pd.DataFrame({'ID': [s.split('.')[0] for s in os.listdir('test/test')  if s.split('.')[1] != 'jfif']})
for i, c in enumerate(learn.data.classes):
  submission[c] = preds_test[:,i]
submission = submission[['ID', 'leaf_rust', 'stem_rust', 'healthy_wheat']] # Get same order as sample sub
submission.head()

,ID,leaf_rust,stem_rust,healthy_wheat
0,NGZ2X2,3.837716e-09,0.999812,1.878243e-04
1,AD54KD,1.313740e-04,0.997699,2.169824e-03
2,3UZ447,1.224209e-04,0.898686,1.011912e-01
3,NN5F6I,9.999516e-01,0.000024,2.450330e-05
4,LE4WVZ,9.998279e-01,0.000171,8.014900e-07


In [0]:
submission.to_csv('submission4.csv', index = False)